## SILVER TO GOLD LAYER


In [11]:
import pandas as pd
from global_paths import (
    TradeHistorySilverFilePath,
    TradeHistoryGoldFilePath,
    SymbolSilverFilePath,
)

### Gold Layer - Trade History


In [12]:
# read the csv file
df_TradeHistory = pd.read_csv(TradeHistorySilverFilePath)

# replace scrip code to compnay name
df_Symbol = pd.read_csv(SymbolSilverFilePath)
df_Symbol["scrip_code"] = df_Symbol["scrip_code"].astype(str)

# Convert 'datetime' to datetime type
df_TradeHistory["datetime"] = pd.to_datetime(df_TradeHistory["datetime"])

# Merge df_TradeHistory with df_Symbol on the matching columns
df_TradeHistory = df_TradeHistory.merge(
    df_Symbol[["scrip_code", "symbol"]],
    left_on="scrip_code",
    right_on="scrip_code",
    how="left",
)

# Assign the new column 'stock_name' in df_TradeHistory to the values from 'symbol'
df_TradeHistory["stock_name"] = df_TradeHistory["symbol"].combine_first(
    df_TradeHistory["company"]
)

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by="datetime")

trade_history = dict()
data = list()

for _, row in df_TradeHistory.iterrows():
    stock_name = row["stock_name"]
    if stock_name not in trade_history:
        trade_history[stock_name] = {"price": 0, "quantity": 0}

    if str(row["trade_type"]).upper() == "BUY":
        row["buy_price"] = row["trade_price"]
        row["buy_quantity"] = row["trade_quantity"]
        row["buy_amount"] = row["trade_price"] * row["trade_quantity"]

        trade_history[stock_name]["price"] = (
            (trade_history[stock_name]["price"] * trade_history[stock_name]["quantity"])
            + (row["trade_price"] * row["trade_quantity"])
        ) / (trade_history[stock_name]["quantity"] + row["trade_quantity"])
        trade_history[stock_name]["quantity"] += row["trade_quantity"]

    elif str(row["trade_type"]).upper() == "SELL":
        row["sell_price"] = row["trade_price"]
        row["sell_quantity"] = row["trade_quantity"]
        row["sell_amount"] = row["trade_price"] * row["trade_quantity"]

        row["buy_price"] = trade_history[stock_name]["price"]
        row["buy_amount"] = row["trade_price"] * trade_history[stock_name]["price"]

        row["profit_loss_amount"] = (
            row["trade_price"] - trade_history[stock_name]["price"]
        ) * row["trade_quantity"]
        trade_history[stock_name]["quantity"] -= row["trade_quantity"]

        # Update the average price only if there is remaining quantity
        if trade_history[stock_name]["quantity"] == 0:
            trade_history[stock_name]["price"] = 0
        elif trade_history[stock_name]["quantity"] < 0:
            print("check for this might be wrong", stock_name, row["datetime"])
            trade_history[stock_name]["price"] = (
                (
                    trade_history[stock_name]["price"]
                    * trade_history[stock_name]["quantity"]
                )
                - (row["trade_price"] * row["trade_quantity"])
            ) / (trade_history[stock_name]["quantity"] - row["trade_quantity"])
        else:
            pass

    else:
        raise Exception(f'{row["trade_type"]} was never excepected')

    row["holding_quantity"] = trade_history[stock_name]["quantity"]
    row["holding_price_avg"] = trade_history[stock_name]["price"]
    row["holding_amount"] = (
        trade_history[stock_name]["price"] * trade_history[stock_name]["quantity"]
    )
    data.append(row)

df_TradeHistory = pd.DataFrame(data)
df_TradeHistory = df_TradeHistory.round(2)

df_TradeHistory = df_TradeHistory[
    [
        "datetime",
        "segment",
        "stock_name",
        "trade_type",
        "buy_quantity",
        "buy_price",
        "buy_amount",
        "sell_quantity",
        "sell_price",
        "sell_amount",
        "profit_loss_amount",
        "holding_quantity",
        "holding_price_avg",
        "holding_amount",
    ]
]

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by=["segment", "stock_name", "datetime"])

# Save the result as a csv file
df_TradeHistory.to_csv(TradeHistoryGoldFilePath, index=None)
df_TradeHistory.info()
print("Gold Layer csv file for trade history successfully created at:")
print(TradeHistoryGoldFilePath.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 166 entries, 2 to 93
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            166 non-null    datetime64[ns]
 1   segment             166 non-null    object        
 2   stock_name          166 non-null    object        
 3   trade_type          166 non-null    object        
 4   buy_quantity        88 non-null     float64       
 5   buy_price           166 non-null    float64       
 6   buy_amount          166 non-null    float64       
 7   sell_quantity       78 non-null     float64       
 8   sell_price          78 non-null     float64       
 9   sell_amount         78 non-null     float64       
 10  profit_loss_amount  78 non-null     float64       
 11  holding_quantity    166 non-null    float64       
 12  holding_price_avg   166 non-null    float64       
 13  holding_amount      166 non-null    float64       
dtype